<a href="https://colab.research.google.com/github/anubhavgupta1/Dive-Into-Deep-Learning/blob/main/Deep%20Learning%20Computation/GPU%20Management/mxnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Deep Learning Computation

In [ ]:
!pip install -U mxnet-cu101==1.7.0

In [2]:
!nvidia-smi

Tue Jan 26 13:00:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from mxnet import np, npx
from mxnet.gluon import nn
npx.set_np()

### Computing Devices

In [4]:
npx.cpu()

cpu(0)

In [5]:
npx.gpu()

gpu(0)

In [6]:
npx.gpu(1)

gpu(1)

In [7]:
def try_gpu(i=0):  #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    return npx.gpu(i) if npx.num_gpus() >= i + 1 else npx.cpu()

In [8]:
try_gpu()

gpu(0)

In [9]:
npx.num_gpus()

1

In [10]:
def try_all_gpus():  #@save
    """Return all available GPUs, or [cpu()] if no GPU exists."""
    devices = [npx.gpu(i) for i in range(npx.num_gpus())]
    return devices if devices else [npx.cpu()]

In [11]:
 try_all_gpus()

[gpu(0)]

### Storage on the GPU

In [12]:
import time

In [13]:
%%time
X = np.ones((1000,1000), ctx=try_gpu())
Y = np.random.uniform(size=(1000, 1000), ctx=try_gpu(1))

CPU times: user 2.59 s, sys: 1.04 s, total: 3.62 s
Wall time: 7.08 s


In [14]:
%%time
Z = X.copyto(try_gpu(1))

CPU times: user 7.66 ms, sys: 5.22 ms, total: 12.9 ms
Wall time: 14 ms


In [15]:
%%time
Y + Z

CPU times: user 1.16 ms, sys: 2.07 ms, total: 3.23 ms
Wall time: 3.3 ms


array([[1.5488136, 1.5928446, 1.7151893, ..., 1.8880436, 1.5058664,
        1.7086089],
       [1.3103808, 1.2777736, 1.3730348, ..., 1.5879672, 1.6771412,
        1.8979559],
       [1.5928802, 1.9202718, 1.0100636, ..., 1.1494986, 1.227741 ,
        1.6995358],
       ...,
       [1.2827585, 1.9108273, 1.2344983, ..., 1.5436208, 1.7582824,
        1.6928148],
       [1.5432006, 1.5367162, 1.3960181, ..., 1.7336426, 1.8024921,
        1.8153074],
       [1.0489427, 1.1327673, 1.1755751, ..., 1.7524731, 1.265617 ,
        1.7676003]])

In [16]:
Z.as_in_ctx(try_gpu(1)) is Z

True

### Neural Networks and GPUs

In [17]:
net = nn.Sequential()
net.add(nn.Dense(1))
net.initialize(ctx=try_gpu())

In [ ]:
net(X)

In [19]:
## Let us confirm that the model parameters are stored on the same GPU.
net[0].weight.data().ctx

gpu(0)